# Astronomy 8824 - Problem Set 4
The goal of this problem set is to gain familiarity with multivariate Gaussians and Markov Chain Monte Carlo

This problem set was developed by David Weinberg, with some modifications by Paul Martini.

In [1]:
# Install the corner module to make some of the plots
! pip install corner --user

In [2]:
import numpy as np
%matplotlib inline
from numpy import matrix
from numpy import linalg
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import corner

# matplotlib settings 
SMALL_SIZE = 14
MEDIUM_SIZE = 16
BIGGER_SIZE = 18

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
plt.rc('lines', linewidth=2)
plt.rc('axes', linewidth=2)
plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)   # fontsize of the figure title

## 1. Bivariate Gaussian

Generate 5000 random data pairs (p1, p2) where p1 and p2 are drawn independently from Gaussians of standard deviation σ1 = 2 and σ2 = 0.5, respectively (with mean zero). Use the python routine np.random.normal().

Compute new data pairs (x, y) with x = p1 cos α − p2 sin α and y = p1 sin α + p2 cos α for α = π/6.

Plot these two distributions (e.g., as tiny dots with different colors) over the top of each other, and plot as x- and y-axis histograms the marginal distributions of p1, p2, x, and y.

Using equations 3.85 - 3.87 of Ivezic et al., compute the expected values of $\sigma_X$, $\sigma_Y$, and $\sigma_{XY}$. Are the marginal distributions for x and y in your plot Gaussians with the expected widths? Overplot Gaussians for comparison.

What is the covariance matrix of (x, y)? Compute this analytically, though it may be useful to compare it to your numerical estimate.

Draw 5000 random data pairs from a bivariate Gaussian with this covariance matrix using np.random.multivariate_normal().

Compare this distribution and the marginal distributions of x and y to the ones you got by your previous procedure and comment on the result.

In [3]:
### Here are some starting routines for the plots

def gaussian(x, mu, sig):
    '''
    calculate a gaussian with mean mu and dispersion sig at input points x
    '''
    return np.exp( -0.5 * np.power( (x - mu)/sig, 2.)) / (np.sqrt(2*np.pi)*sig)

def PlotTwoDist(xy1, xy2, label1, label2, size, addgauss=False, gxsig=False, gysig=False, connect=20): 
    '''
    xy1, xy2: (x,y) points for the two distributions
    label1, label2: labels for the two distributions
    size: (float) half width of plot range
    addgauss: (bool) True to overplot Gaussian on each histogram
    gxsig, gysig: (float) sigma values for the two histograms
    '''
    bins = np.arange(-1.*size, size, 0.25)
    fig = plt.figure(figsize=(8,8))
    gs = GridSpec(4,4)
    ax_scatter = fig.add_subplot(gs[1:4, 0:3])
    ax_xhist = fig.add_subplot(gs[0,0:3])
    ax_yhist = fig.add_subplot(gs[1:4, 3])
    ax_scatter.scatter(xy1[0], xy1[1], color='k', s=1, label=label1)
    ax_scatter.scatter(xy2[0], xy2[1], color='r', s=1, label=label2)
    ax_scatter.plot(xy2[0][:connect], xy2[1][:connect], color='r', ls='-')
    ax_xhist.hist(xy1[0], bins=bins, histtype='step', color='k', density=True)
    ax_xhist.hist(xy2[0], bins=bins, histtype='step', color='r', density=True)
    ax_yhist.hist(xy1[1], bins=bins, histtype='step', color='k', density=True, orientation='horizontal')
    ax_yhist.hist(xy2[1], bins=bins, histtype='step', color='r', density=True, orientation='horizontal')

    if addgauss and gxsig and gysig: 
        gg = np.linspace(-1*size, size, 100)
        ggx = gaussian(gg, 0., gxsig)
        ggy = gaussian(gg, 0., gysig)
        ax_xhist.plot(gg, ggx)
        ax_yhist.plot(ggy, gg)
    
    plt.setp(ax_xhist.get_xticklabels(), visible=False)
    plt.setp(ax_xhist.get_yticklabels(), visible=False)
    plt.setp(ax_yhist.get_xticklabels(), visible=False)
    plt.setp(ax_yhist.get_yticklabels(), visible=False)
    ax_scatter.set_xlabel("X", fontsize=16)
    ax_scatter.set_ylabel("Y", fontsize=16)
    ax_scatter.set_xlim(-8, 8)
    ax_scatter.set_ylim(-8, 8)
    ax_xhist.set_ylabel("N", fontsize=16)
    ax_yhist.set_xlabel("N", fontsize=16)
    ax_scatter.legend(frameon=False, fontsize=16)

In [4]:
### Here is a start for the first part of the problem set

# Generate bivariate Gaussian distributions following Ivezic et al. sections 3.5.2 and 3.5.4
nelem = 5000
seed = 12         # starting seed for random number generator

# means and dispersions of initial Gaussians
mux = 0
muy = 0
sig1 = 2
sig2 = 0.5

# Generate (p1, p2) with np.random.normal()
np.random.seed(seed)
p1 = sig1*np.random.normal(size=nelem)
p2 = sig2*np.random.normal(size=nelem)

# Compute (x, y) from (p1, p2) rotated by angle alpha
alpha = np.pi/6
cosa = np.cos(alpha)
sina = np.sin(alpha)
x = ...
y = ...

# Compute the expected values of the variances of x and y and the covariance xy
# (See Ivezic et al. eqns 3.85 and 3.86)
sigx2 = ...
sigy2 = ...
sigxy = ...

# Use np.random.multivariate with the mean and expected covariance matrix
# Ivezic et al. eqs. 3.85-3.87
mu = ...
cov = ...
# print ("Covariance Matrix = ", cov)
# xy=np.random.multivariate_normal(mu,cov,size=nelem)

## 2. MCMC realization of a 2-d probability distribution

The probability distribution for the bivariate Gaussian distribution in Part 1 is:
$$
p(\vec x) =  {1 \over 2\pi\sqrt{{\rm det}({\bf C})}}
  \exp\left(-{1\over 2} \vec x^T {\bf H} \vec x\right),
$$
where $\vec x=(x,y)$, ${\bf C} = \left({\sigma_x^2 \atop \sigma_{xy}} {\sigma_{xy} \atop \sigma_y^2}\right)$, and ${\bf H} = {\bf C}^{-1}$.

Implement a simple Markov Chain Monte Carlo (MCMC) routine:

1. Start at a user-specified location $x_0,y_0$.

2. At each iteration generate a trial point $(x_{i+1},y_{i+1})$ with
$$\eqalign{
x_{i+1}&=x_i + h\sigma_x {\cal N}(0,1)  \cr
y_{i+1}&=y_i + h\sigma_y {\cal N}(0,1)  \cr
}
$$
where ${\cal N}(0,1)$ is a Gaussian random variable of zero mean and unit dispersion (chosen separately for $x$ and $y$) and $h$ is a user-specified scaling of step size.

3. If $p(x_{i+1},y_{i+1}) > p(x_i,y_i)$ accept the step, i.e., add the new pair to the chain and take your next trial step from this new position.

4. If $p(x_{i+1},y_{i+1}) < p(x_i,y_i)$ then accept the step with probability $p(x_{i+1},y_{i+1})/p(x_i,y_i)$ (draw a uniform random deviate and compare it to this ratio).  If the step is not accepted, add the previous point ($x_i, y_i$) to the chain and generate a new trial point.

5. Output the final distribution of the chain.  Also keep track of and report the fraction of trial steps that are accepted, i.e., the ratio of the final length of the chain to the total number of steps needed to produce it.

In [5]:
# Starting point

# DHW mc2d.py, modified for notebook 

def MultiGaussProb(x,cinv,prefac):
    '''
    Return multivariate Gaussian probability
    x: vector of data values(matrix)
    cinv: inverse covariance matrix
    prefac:  prefactor for properly normalized Gaussian (float),
             taken as input so that it isn't computed every call
             should be [(2\pi)^{M/2} \sqrt{det(C)}]^{-1}
    '''
    arg = float(x * cinv * x.T)
    return (prefac * np.exp(-arg/2.))

def mcmc(xy_start, xy_step, nchain, nthin, seed, sig1=2, sig2=0.5):
    '''
    xy_start: [xstart, ystart]
    xy_step: [xstep, ystep]
    nchain: output chain length (after thinning)
    nthin: thinning factor
    '''
    np.random.random(seed)
    
    # generate a chain nthin times longer than the final chain
    nchain = nthin * nchain    
    chain=np.zeros((nchain,3))    # store (x,y,p) as elements of chain
    
    mux = 0
    muy = 0
    alpha= np.pi/6
    cosa = np.cos(alpha)
    sina = np.sin(alpha)
    
    '''
    your code here
    '''
    
    # Output the chain 'thinned' by the factor nthin
    return chain[::nthin]

### 2.1 Use this program to generate a 5000-element chain starting from $(x,y)=(1,1)$ with step scaling $h=1$.  

Plot the distribution of points from this chain, and the corresponding marginal distributions, over the bivariate Gaussian distribution from Part 1. If your programs are working, you should get good agreement.

In [6]:
### Answer

xy_start = np.array([1.,1.])
xy_step = np.array([1.,1.])
# chain1 = mcmc(xy_start, xy_step, 5000, 1, 12)

# Illustration of how to use PlotTwoDist() -- 
# PlotTwoDist(xy.T, chain1.T, "Gaussian", "MCMC", 8, connect=20)

### 2.2 Try several different starting points and compare the results. You can just describe this comparison in words. 

In [7]:
### Answer 

### 2.2 Change h from 1 to 0.1 and plot the results. Compare the distribution to that for h = 1, and comment on the fraction of steps that are accepted.

In [8]:
### Answer 

### 2.3 Change h from 1 to 2.5. Compare the distribution to that for h = 1 (with a plot), and compare the fraction of steps that are accepted.

In [9]:
### Answer 

### 2.4 Change $\sigma_2$ from 0.5 to 0.1. Compare to your previous results for h = 1.

In [10]:
### Answer 

### 2.5 Comment on issues of efficiency and accuracy in MCMC computations and strategies that could improve the efficiency for the $\sigma_2 = 0.1$ case.

### Answer

# 3. Cosmic MCMC: Parameters of the Universe

$\newcommand{\hubunits}{\,\hbox{km s$^{-1}$ Mpc$^{-1}$}}$

Here we will do a simplified version of the statistical analysis in Aubourg et al. (2015).

You may again use cosmodist_subs.py or astropy. If you used cosmodist_subs.py, use the (new) routine comsmodisth(), which returns different quantities than cosmodist(). In the latter case, I recommend adopting a tolerance of $3\times 10^{-5}$, which is adequate given the uncertainties of our observational constraints, and your code will evaluate this integral many times. 

This time, we will use its ability to compute distances for $\Omega_k \neq 0$  and $w=-1$.  Refer back to PS 3 for the relevant equations. 

As the cosmological constraints, take the following (the first two are from the CMB, and others are from BAO measurements):

$\Omega_m h^2 = 0.1386 \pm 0.0027.$

$D_M(z=1090) = 13962 \pm 10$ Mpc.

$D_M(z=2.34) = 5381 \pm 170$ Mpc.

$H(z=2.34) = 222 \pm 5 \hubunits.$

$D_M(z=0.57) = 2204 \pm 31$ Mpc.

$H(z=0.57) = 98 \pm 3 \hubunits.$

$D_M(z=0.32) = 1249 \pm 25$ Mpc.

Compute the likelihood of the data for a given set of cosmological parameters as $L \propto e^{-\chi^2/2}$, where $\chi^2$ is computed from the above data values ignoring any error covariances (i.e., $\chi^2 = \sum (y_i-y_{{\rm mod},i})^2/\sigma_i^2$). 

Adapt your MCMC code to create a chain for cosmological parameter values. You should set it up to allow steps in 4 parameters: $\Omega_m$, $h$, $w$, and $\Omega_k$. 

I recommend you write it to take as input a set of starting values (e.g. a dictionary called 'startvals') and a set of step sizes (e.g. a dictionary called 'stepvals'), and set the stepsize equal to zero for any parameter that should be held fixed. You should also use $e^{-\Delta\chi^2/2}$ to compute the ratio of the probabilities.

In [11]:
h0 = 70. 
om = 0.3
ok = 0. 
w = -1
zrec = 1090.
tcmb = 2.726

# Illustration of the use of cosmodist_subs -- 

import cosmodist_subs as cs

print("Timing of cs.cosmodisth()")
%timeit  cs.cosmodisth(1090, h0, om, ok, w)

[dmrec, hzrec] = cs.cosmodisth(1090, h0, om, ok, w)
print("cs.cosmodisth() =", dmrec, hzrec)

# Illustration of the use of astropy -- 

from astropy.cosmology import FlatLambdaCDM

cosmo = FlatLambdaCDM(H0=h0, Om0=om, Tcmb0=tcmb)

print("Timing of astropy.cosmology.FlatLambdaCDM")
%timeit FlatLambdaCDM(H0=h0, Om0=om, Tcmb0=tcmb), cosmo.comoving_distance(zrec).value, cosmo.H(zrec).value/h0

print("astropy output =", cosmo.comoving_distance(zrec).value, cosmo.H(zrec).value/h0)

print("Note the significant speed up with astropy!")

Timing of cs.cosmodisth()
2.84 ms ± 26.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
cs.cosmodisth() = 13615.917774798161 22594.79521294432
Timing of astropy.cosmology.FlatLambdaCDM
436 µs ± 1.13 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
astropy output = 13615.922648878024 22594.96028133528
Note the significant speed up with astropy!


In [12]:
# Starter code

# DHW cosmomc.py modified for notebook
def prob(model,data,errors):
    """
    Return e^{-chi^2/2}, for diagonal covariance matrix
    model = vector of model predictions
    data = vector of data values
    errors = vector of errors on data
    """
    dev = (model - data)/errors
    return (np.exp(-1.*sum(dev**2)/2.))

def modelvals(om,h,w,ok,astropy=True):
    """
    Return vector of the model predicted data values
    """
    omhsqr=om*(h**2)
    h0=100.*h                                   # value in km/s/Mpc
    if astropy: 
        cosmo = FlatLambdaCDM(H0=h0, Om0=om, Tcmb0=tcmb)
        [dmzrec,hzrec] = cosmo.comoving_distance(zrec).value, cosmo.H(zrec).value/h0
        [dmz234,hz234] = cosmo.comoving_distance(2.34).value, cosmo.H(2.34).value/h0
        [dmz057,hz057] = cosmo.comoving_distance(0.57).value, cosmo.H(0.57).value/h0
        [dmz032,hz032] = cosmo.comoving_distance(0.32).value, cosmo.H(0.32).value/h0       
    else: 
        [dmzrec,hzrec] = cs.cosmodisth(1090, h0, om, ok, w)
        [dmz234,hz234] = cs.cosmodisth(2.34, h0, om, ok, w)
        [dmz057,hz057] = cs.cosmodisth(0.57, h0, om, ok, w)
        [dmz032,hz032] = cs.cosmodisth(0.32, h0, om, ok, w)

    return (np.array([omhsqr,dmzrec,dmz234,hz234*h0,dmz057,hz057*h0,dmz032]))

def cosmomcmc(startvals, stepvals, nchain, nthin, seed, astropy=True):
    
    omstart = startvals['omega_m']
    hstart = startvals['H0']
    okstart = startvals['omega_k']
    wstart = startvals['w']
    
    omstep = stepvals['omega_m']
    hstep = stepvals['H0']
    okstep = stepvals['omega_k']
    wstep = stepvals['w']
    
    np.random.seed(seed)

    nchain = nchain * nthin     # number of elements needed for nchain outputs
    
    # data values and errors being used
    omhsqr=0.1386
    omhsqrerr=0.0027
    dmzrec=13962
    dmzrecerr=10
    dmz234=5381
    dmz234err=170
    hz234=222
    hz234err=5
    dmz057=2204
    dmz057err=31
    hz057=98
    hz057err=3
    dmz032=1249
    dmz032err=25

    data = np.array([omhsqr,dmzrec,dmz234,hz234,dmz057,hz057,dmz032])
    errors = np.array([omhsqrerr,dmzrecerr,dmz234err,hz234err,dmz057err,hz057err,dmz032err])

    # We are going to save all elements of the chain in memory to make
    # the i/o more efficient (one np.savetxt at the end), but if memory
    # were an issue we could write out steps one at a time (or in chunks)
    chain = np.zeros((nchain,5))
    chain[0][0] = om1 = omstart
    chain[0][1] = h1 = hstart
    chain[0][2] = w1 = wstart
    chain[0][3] = ok1 = okstart

    '''
    Your code here
    '''

### 3.1 Flat Universe

First consider a flat universe, with $\Omega_k=0$, but allowing free $w$. Create a 3-d MCMC chain using the parameters $\Omega_m$, $h$, and $w$, where $h \equiv H_0/100\hubunits$. Use your 4-d code but set the step size in the $\Omega_k$ dimension to zero. For a starting point I suggest $\Omega_m=0.3$, $h=0.68$, $w=-1$, and for initial step sizes I suggest trying $\Delta = 0.03$ in each parameter. 

I recommend you use a small (e.g. 2000) number of points for debugging and then a larger number (10-50k) when you are ready to make your final plot. 

Note that $\Delta$ here refers to the actual steps in $\Omega_m$, $h$, and $w$, and I have chosen it because I know
that these data give parameter errors that are roughly in this ballpark. Do not further multiply 0.03 by the expected standard deviations of these parameters --- that would be like taking $h=0.03$ in Part 2, and you already saw (I hope) that $h=0.1$ leads to chains that don't explore the likelihood surface very well.  I warn you in advance that with $\Delta = 0.03$ your acceptance fraction in the MCMC will be low ($\sim 1\%$), but if you take a much smaller step then you will not get good likelihood sampling.

Plot the distribution of your points in the planes $w$ vs. $\Omega_m$, $w$ vs. $h$, and $\Omega_m$ vs. $h$ with [corner.py](https://corner.readthedocs.io/en/latest/).

For reference, you may want to look at Figure 8 of Aubourg et al. (the $w$CDM panel for 3.1 and the o$\Lambda$CDM panel for 3.2), but you shouldn't expect to get exactly the same results. The main simplifications are that you are not including covariances of the errors and that I have converted the BAO measurements to absolute units using the best-fit value of the sound horizon $r_d$, which is well known (to 0.4\%) but not perfectly known; a full calculation would consider its dependence on cosmological parameters.

In [13]:
startvals = dict()
startvals['omega_m'] = 0.3
startvals['H0'] = 0.68
startvals['omega_k'] = 0.
startvals['w'] = -1.

stepvals = dict()
stepvals['omega_m'] = 0.03
stepvals['H0'] = 0.03
stepvals['omega_k'] = 0.0 # hold fixed
stepvals['w'] = 0.03

# c_chain31 = cosmomcmc(startvals, stepvals, nchain=4000, nthin=1, seed=12, astropy=True)
# c_chain31 = cosmomcmc(startvals, stepvals, nchain=5000, nthin=50, seed=12, astropy=True)

In [14]:
# Create the corner plot

# Illustration of how to use corner()
# corner_input31 = c_chain31.T[0:3].T
# figure = corner.corner(corner_input31, labels=[r"$\Omega_m$", r"$h$", r"$w$"],
#                      quantiles=[0.5], 
#                      show_titles=True, title_kwargs={"fontsize": 14}, title_fmt='.2f',
#                      label_kwargs={"fontsize": 14},
#                      hist2d_kwargs={"plot_contours": False, "plot_density": False}
#                      )

### 3.2 Lambda Universe

Now consider a universe, with $w=-1$ and free $\Omega_k$. Create a 2000-point, 3-d MCMC using the parameters 
$\Omega_m$, $h$, and $\Omega_k$. (Set the step size in the $w$ dimension to zero.) I suggest you start with $\Omega_m=0.3$, $h=0.68$, $\Omega_k=0$, and for initial step sizes I suggest trying $\Delta = 0.03$ in the first two parameters and $\Delta = 0.003$ in $\Omega_k$.

Plot the distribution of your points in the planes $\Omega_k$ vs. $\Omega_m$, $\Omega_k$ vs. $h$, and $\Omega_m$ vs. $h$ with the corner module.

In [15]:
### Answer 